In [1]:
import cv2
import numpy as np
import tensorflow as tf

In [2]:
class EAST:
    def __init__(self) -> None:
        self.tfsession = None
        self.graph = None
        self.graph_def = None
        self.image = None

    def load_east_model(self, model_path) -> None:
        # Load the EAST model from the provided path
        with tf.compat.v1.Session() as sess:
            # Load the model
            saver = tf.compat.v1.train.import_meta_graph(model_path + '.meta')
            saver.restore(sess, model_path)

            # Get the graph definition
            

            self.tfsession = sess
            self.graph = tf.compat.v1.get_default_graph()
            self.graph_def = self.graph.as_graph_def()
    
    def load_image(self, imagepath) -> None:
        self.image = cv2.imread(imagepath)
    
    def print_model_nodes(self) -> None:
        # Print the names of input and output nodes
        input_nodes = [node.name for node in self.graph_def.node if node.op == 'Placeholder']
        output_nodes = [node.name for node in self.graph_def.node if 'Sigmoid' in node.name]  # Adjust this condition based on your actual output node names

        print("Input Nodes:")
        for input_node in input_nodes:
            print(input_node)

        print("\nOutput Nodes:")
        for output_node in output_nodes:
            print(output_node)

    def east_text_detection(self, conf_threshold=0.5):
        # Get the input and output tensors from the graph
        input_tensor = self.graph.get_tensor_by_name('input_images:0')
        output_tensors = {
            'scores': self.graph.get_tensor_by_name('model_0/feature_fusion/Conv_7/Sigmoid:0'),
            'geometry': self.graph.get_tensor_by_name('model_1/feature_fusion/Conv_7/Sigmoid:0')
        }

        # Get image dimensions
        height, width = self.image.shape[:2]

        # Preprocess the image for EAST
        image = cv2.resize(self.image, (320, 320))  # Resize the image to match the model's input size
        image = image / 255.0  # Normalize pixel values to the range [0, 1]
        image = np.expand_dims(image, axis=0)  # Add batch dimension

        # Perform inference
        scores, geometry = self.tfsession.run([output_tensors['scores'], output_tensors['geometry']], feed_dict={input_tensor: image})

        # Decode the predictions
        rectangles, confidences = self.decode_predictions(scores, geometry, conf_threshold)

        # Return the detected rectangles and their confidences
        return rectangles, confidences

    def decode_predictions(self, scores, geometry, conf_threshold):
        # Get the height, width, and number of scores
        height, width = scores.shape[2:4]
        num_scores = 5

        rectangles = []
        confidences = []

        for y in range(0, height):
            scores_data = scores[0, 0, y]
            x_data0 = geometry[0, 0, y]
            x_data1 = geometry[0, 1, y]
            x_data2 = geometry[0, 2, y]
            x_data3 = geometry[0, 3, y]
            angles_data = geometry[0, 4, y]

            for x in range(0, width):
                score = scores_data[x]

                if score < conf_threshold:
                    continue

                offset_x = x * 4.0
                offset_y = y * 4.0

                angle = angles_data[x]
                cos_a = np.cos(angle)
                sin_a = np.sin(angle)

                h = x_data0[x] + x_data2[x]
                w = x_data1[x] + x_data3[x]

                offset = np.array([offset_x + cos_a * x_data1[x] + sin_a * x_data3[x],
                                offset_y - sin_a * x_data1[x] + cos_a * x_data3[x]])

                p1 = (-sin_a * offset_y + cos_a * offset_x - w // 2, cos_a * offset_y + sin_a * offset_x - h // 2)
                p2 = (-sin_a * offset_y + cos_a * offset_x + w // 2, cos_a * offset_y + sin_a * offset_x - h // 2)
                p3 = (-sin_a * offset_y + cos_a * offset_x + w // 2, cos_a * offset_y + sin_a * offset_x + h // 2)
                p4 = (-sin_a * offset_y + cos_a * offset_x - w // 2, cos_a * offset_y + sin_a * offset_x + h // 2)

                rectangles.append((p1, p2, p3, p4))
                confidences.append(float(score))

        return rectangles, confidences

    def draw_predictions(self, image, rectangles, confidences):
        # Draw the rectangles and confidences on the image
        for i in range(len(rectangles)):
            box = np.int0(rectangles[i])
            confidence = confidences[i]
            cv2.drawContours(image, [box], 0, (0, 255, 0), 2)
            cv2.putText(image, f'{confidence:.2f}', (box[0][0], box[0][1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Display the image
        cv2.imshow('Text Detection', image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [3]:
east_model_path = 'eastmodel/model.ckpt-49491'

In [4]:
east = EAST()
east.load_east_model(east_model_path)

INFO:tensorflow:Restoring parameters from eastmodel/model.ckpt-49491


In [5]:
east.print_model_nodes()

Input Nodes:
input_images
input_score_maps
input_geo_maps
input_training_masks

Output Nodes:
model_0/feature_fusion/Conv_7/Sigmoid
model_0/feature_fusion/Conv_8/Sigmoid
model_0/feature_fusion/Conv_9/Sigmoid
model_0/gradients/model_0/feature_fusion/Conv_7/Sigmoid_grad/SigmoidGrad
model_0/gradients/model_0/feature_fusion/Conv_8/Sigmoid_grad/SigmoidGrad
model_0/gradients/model_0/feature_fusion/Conv_9/Sigmoid_grad/SigmoidGrad
model_1/feature_fusion/Conv_7/Sigmoid
model_1/feature_fusion/Conv_8/Sigmoid
model_1/feature_fusion/Conv_9/Sigmoid
model_1/gradients/model_1/feature_fusion/Conv_7/Sigmoid_grad/SigmoidGrad
model_1/gradients/model_1/feature_fusion/Conv_8/Sigmoid_grad/SigmoidGrad
model_1/gradients/model_1/feature_fusion/Conv_9/Sigmoid_grad/SigmoidGrad


In [7]:
east.load_image('azure2.jpg')

In [8]:
east.east_text_detection()

RuntimeError: Attempted to use a closed Session.